# 2팀 RFP RAG 시스템 중간발표
## 기술 구현 코드 설명 

---

## 1. 시스템 아키텍처 - 문서 처리 모듈

### PDF 파일 처리: PyMuPDF 라이브러리를 활용한 텍스트 추출
라이브러리는 빠른 처리 속도와 높은 텍스트 추출 정확도를 제공하며, 복잡한 PDF 레이아웃에서도 안정적으로 텍스트를 추출할 수 있습니다. 페이지별로 순차 처리하여 전체 문서의 텍스트 콘텐츠를 획득합니다.

In [ ]:
# src/processors/pdf_processor.py
class PDFProcessor(DocumentProcessor):
    def extract_content(self, file_path: str) -> Dict[str, Any]:
        """PDF에서 텍스트, 이미지, 테이블 등을 추출"""
        content = {
            'text': '',
            'images': [],
            'tables': [],
            'metadata': self._create_base_metadata(file_path)
        }

        # PyMuPDF로 텍스트 추출
        doc = fitz.open(file_path)
        full_text = ""
        for page in doc:
            page_text = page.get_text()
            full_text += page_text + "\n"

        content['text'] = full_text
        return content

### HWP 파일 처리: LibreOffice 헤드리스 모드를 통한 변환 처리
공공기관에서 주로 사용하는 HWP 파일을 처리하기 위해 LibreOffice의 헤드리스 모드를 활용합니다. HWP 파일을 먼저 DOCX 형식으로 변환한 후, python-docx 라이브러리를 사용해 텍스트를 추출합니다. 이 방식을 통해 한글 문서의 복잡한 서식을 유지하면서도 안정적인 텍스트 추출이 가능합니다.

In [ ]:
# src/processors/hwp_processor.py
class HWPProcessor(DocumentProcessor):
    def extract_content(self, file_path: str) -> Dict[str, Any]:
        """HWP 파일에서 텍스트와 이미지 추출"""
        # LibreOffice를 통한 HWP 변환
        converted_file = self._convert_hwp_to_docx(file_path)

        # 텍스트 추출
        doc = Document(converted_file)
        text_content = ""
        for paragraph in doc.paragraphs:
            text_content += paragraph.text + "\n"
            
        return {
            'text': text_content,
            'metadata': self._create_base_metadata(file_path)
        }

### 스마트 청킹: 1000자 단위 오버랩 방식으로 검색 성능 최적화
문서를 검색에 적합한 크기로 분할하기 위해 1000자 단위 청킹을 수행합니다. 청크 간 200자 오버랩을 적용하여 문맥이 끊어지는 것을 방지하고, 각 청크에는 고유 UUID와 문서 ID를 부여해 추적 가능성을 보장합니다. 이런 방식으로 총 1,774개의 검색 가능한 텍스트 청크를 생성했습니다.

In [ ]:
# src/processors/base.py
def _chunk_by_size(self, text: str, metadata: Dict[str, Any]) -> List[DocumentChunk]:
    """크기 기반 청킹 - 1000자 단위"""
    chunks = []
    text_length = len(text)

    # chunk_size = 1000, overlap = 200
    for i in range(0, text_length, self.chunk_size - self.overlap):
        chunk_text = text[i:i + self.chunk_size]

        if chunk_text.strip():  # 빈 청크 제외
            chunk = DocumentChunk(
                content=chunk_text.strip(),
                metadata=metadata.copy(),
                chunk_id=str(uuid.uuid4()),
                document_id=metadata.get('document_id'),
                chunk_index=len(chunks)
            )
            chunks.append(chunk)

    return chunks

### 멀티모달 처리: GPT-4V를 활용한 이미지 내 텍스트 및 차트 분석
문서 내 이미지, 표, 차트에 포함된 정보까지 활용하기 위해 GPT-4V 모델을 도입했습니다. 이미지를 Base64로 인코딩하여 OpenAI Vision API에 전송하고, 한국어로 상세한 설명을 요청합니다. 이를 통해 텍스트만으로는 놓칠 수 있는 시각적 정보까지 검색 대상에 포함시켜 더욱 포괄적인 문서 분석이 가능합니다.

In [ ]:
# src/processors/multimodal_processor.py
def analyze_image_with_gpt4v(self, image_path: str) -> str:
    """gpt4 이미지 분석"""
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    response = self.client.chat.completions.create(
        model="gpt-4o",  
        messages=[{
            "role": "user",
            "content": [
                {
                    "type": "text", 
                    "text": "이 이미지의 내용을 한국어로 상세히 설명해주세요."
                },
                {
                    "type": "image_url", 
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                }
            ]
        }],
        max_tokens=300
    )

    return response.choices[0].message.content

---

## 2. 저장소 모듈 - 이중 저장 구조

### ChromaDB 벡터 데이터베이스: OpenAI text-embedding-3-small 모델 기반 의미 검색
벡터 검색을 위해 ChromaDB를 영구 저장소로 구성하고, OpenAI의 text-embedding-3-small 모델을 임베딩 함수로 사용합니다. 이 모델은 1536차원의 고품질 벡터를 생성하여 한국어 문서의 의미적 유사성을 정확하게 포착합니다. PersistentClient를 통해 데이터 영속성을 보장하고, 컬렉션 단위로 문서를 관리합니다.

In [ ]:
# src/storage/vector_store.py
class VectorStore:
    def __init__(self, persist_directory: str = "./vector_db"):
        self.client = chromadb.PersistentClient(path=persist_directory)
        self.collection = self.client.get_or_create_collection(
            name="rfp_documents",
            embedding_function=self._get_embedding_function()
        )

    def _get_embedding_function(self):
       
        return embedding_functions.OpenAIEmbeddingFunction(
            api_key=os.getenv('OPENAI_API_KEY'),
            model_name="text-embedding-3-small"  # 최고 성능 임베딩
        )

    def add_documents(self, chunks: List[DocumentChunk]) -> bool:
        """문서 청크를 벡터 DB에 추가"""
        for chunk in chunks:
            self.collection.add(
                documents=[chunk.content],
                metadatas=[chunk.metadata],
                ids=[chunk.chunk_id]
            )
        return True

### SQLite 메타데이터 저장소: 구조화된 정보 관리 및 빠른 필터링
문서의 구조화된 메타데이터를 효율적으로 관리하기 위해 SQLite 데이터베이스를 사용합니다. documents 테이블에는 발주기관, 예산, 마감일, 사업분야 등 RFP 특화 정보를 저장하고, chunks 테이블에는 각 텍스트 청크의 정보를 저장합니다. 인덱스를 통해 발주기관별, 날짜별 빠른 필터링이 가능하며, 외래키 제약조건으로 데이터 무결성을 보장합니다.

In [ ]:
# src/storage/metadata_store.py
class MetadataStore:
    def _init_database(self):
        """데이터베이스 및 테이블 초기화"""
        # 문서 메타데이터 테이블
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS documents (
                id TEXT PRIMARY KEY,
                file_path TEXT UNIQUE,
                file_name TEXT,
                title TEXT,
                agency TEXT,              -- 발주기관
                budget TEXT,              -- 예산
                deadline TEXT,            -- 마감일
                business_type TEXT,       -- 사업분야
                total_pages INTEGER,
                processed_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        # 청크 정보 테이블
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS chunks (
                chunk_id TEXT PRIMARY KEY,
                document_id TEXT,
                chunk_index INTEGER,
                content_preview TEXT,
                chunk_size INTEGER,
                FOREIGN KEY (document_id) REFERENCES documents (id)
            )
        ''')

---

## 3. 검색 및 답변 모듈 - 하이브리드 검색

### 하이브리드 검색 엔진: 벡터 검색과 BM25 키워드 검색의 가중 조합
두 가지 검색 방식을 결합하여 검색 정확도를 극대화합니다. 벡터 검색은 의미적 유사성을 기반으로 하여 동의어나 유사한 표현도 찾아내고, BM25 키워드 검색은 정확한 용어 매칭을 통해 누락을 방지합니다. 벡터 검색에 70%, 키워드 검색에 30%의 가중치를 적용하여 최적의 검색 결과를 제공합니다.

In [ ]:
# src/retrieval/hybrid_retriever.py
class HybridRetriever:
    def hybrid_search(self, query: str, k: int = 10,
                     vector_weight: float = 0.7,
                     keyword_weight: float = 0.3) -> List[Dict[str, Any]]:
        """하이브리드 검색 수행"""

        # 1. Vector 검색 (의미 기반 검색)
        vector_results = self.vector_store.similarity_search(
            query, k=k*2, filters=filters
        )

        # 2. BM25 키워드 검색 (정확한 용어 매칭)
        keyword_results = []
        if self.document_chunks:
            bm25_scores = self.bm25.search(query, top_k=k*2)
            for idx, score in bm25_scores:
                chunk = self.document_chunks[idx]
                keyword_results.append({
                    'content': chunk.content,
                    'metadata': chunk.metadata,
                    'bm25_score': score
                })

        # 3. 점수 정규화 및 가중 조합
        normalized_results = self._combine_scores(
            vector_results, keyword_results,
            vector_weight, keyword_weight
        )

        return sorted(normalized_results,
                     key=lambda x: x['hybrid_score'], reverse=True)[:k]

### BM25 알고리즘: Okapi BM25 공식을 활용한 통계적 키워드 검색
키워드 검색을 위해 정보 검색 분야의 표준인 Okapi BM25 알고리즘을 구현했습니다. Term Frequency(TF)와 Inverse Document Frequency(IDF)를 조합하여 문서의 관련성을 계산하며, k1=1.5와 b=0.75 파라미터를 통해 용어 빈도 포화도와 문서 길이 정규화를 조정합니다. 이를 통해 정확한 용어 매칭 기반의 검색 결과를 제공합니다.

In [ ]:
# src/retrieval/hybrid_retriever.py
class BM25:
    def __init__(self, k1: float = 1.5, b: float = 0.75):
        self.k1 = k1  # term frequency saturation parameter
        self.b = b    # length normalization parameter

    def search(self, query: str, top_k: int = 10) -> List[Tuple[int, float]]:
        """BM25 점수 계산 및 검색"""
        query_words = query.lower().split()
        scores = []

        for i, doc_freq in enumerate(self.doc_freqs):
            score = 0.0
            for word in query_words:
                if word in doc_freq:
                    tf = doc_freq[word]  # term frequency
                    idf = self.idf_cache.get(word, 0)  # inverse document frequency

                    # BM25 공식
                    numerator = tf * (self.k1 + 1)
                    denominator = tf + self.k1 * (1 - self.b + self.b *
                                                 (self.doc_lengths[i] / self.avg_doc_length))
                    score += idf * (numerator / denominator)

            scores.append((i, score))

        return sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]

---

## 4. GPT-4o 답변 생성 및 신뢰도 계산

### GPT-4o 모델: 컨텍스트 인식 기반 정확한 답변 생성
검색된 문서 컨텍스트를 바탕으로 GPT-4o 모델을 활용해 정확한 답변을 생성합니다. 시스템 프롬프트를 통해 RFP 문서 분석 전문가 역할을 부여하고, temperature=0.1로 설정하여 일관성 있는 답변을 보장합니다. 최대 500토큰으로 제한하여 간결하면서도 필요한 정보를 포함한 답변을 제공합니다.

In [ ]:
# src/rag_system.py
def _generate_openai_answer(self, query: str, context: str) -> str:
    """OpenAI API를 사용한 답변 생성"""
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

    prompt = f"""다음은 RFP(제안요청서) 문서의 관련 내용입니다:

{context}

질문: {query}

위 문서 내용을 바탕으로 정확하고 구체적으로 답변해주세요.
관련 정보가 있다면 구체적으로 설명하고, 정말 관련 정보가 없을 때만
'문서에서 확인할 수 없습니다'라고 답변하세요."""

    response = client.chat.completions.create(
        model="gpt-4o",  # 최신 GPT-4o 모델 사용
        messages=[
            {"role": "system", "content": "당신은 RFP 문서 분석 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1  # 일관성 있는 답변을 위해 낮은 temperature
    )

    return response.choices[0].message.content.strip()

### 신뢰도 기반 할루시네이션 방지: 검색 점수 기반 답변 품질 관리
RAG 시스템의 핵심 과제인 할루시네이션을 방지하기 위해 신뢰도 기반 필터링 시스템을 구현했습니다. 검색 결과의 상위 3개 하이브리드 점수를 평균내어 신뢰도를 계산하고, 기본 임계값 0.3 미만일 경우 "관련 정보를 찾을 수 없다"고 명시적으로 응답합니다. 이를 통해 부정확한 정보 제공을 사전에 차단합니다.

In [ ]:
# src/rag_system.py
def search_and_answer(self, query: str, confidence_threshold: float = 0.3):
    """검색 및 답변 생성 (신뢰도 기반 필터링)"""
    search_results = self.retriever.hybrid_search(query, k=5)
    confidence = self._calculate_confidence(search_results, query)

    # 신뢰도가 임계값 미만이면 검색 실패로 처리
    if confidence < confidence_threshold:
        answer = "문서에서 관련 정보를 찾을 수 없습니다. 다른 키워드로 다시 검색해 보세요."
        answer_confidence = "low"
    else:
        answer = self._generate_answer(query, search_results)
        answer_confidence = "high" if confidence > 0.7 else "medium"

    return {
        'answer': answer,
        'confidence': confidence,
        'answer_confidence': answer_confidence,
        'sources': search_results[:3]
    }

def _calculate_confidence(self, search_results: List[Dict], query: str) -> float:
    """검색 결과의 신뢰도 계산"""
    if not search_results:
        return 0.0

    scores = []
    for result in search_results:
        score = result.get('hybrid_score', result.get('vector_score', 0))
        scores.append(score)

    if scores:
        # 상위 3개 결과의 평균 점수
        top_scores = sorted(scores, reverse=True)[:3]
        confidence = sum(top_scores) / len(top_scores)
        return min(confidence, 1.0)  # 최대 1.0으로 제한

    return 0.0

---

## 5. 웹 인터페이스 - Streamlit 대시보드

### 실시간 시스템 모니터링: 멀티 컬럼 레이아웃과 메트릭 시각화
Streamlit의 st.columns()와 st.metric()을 활용하여 시스템 상태를 실시간으로 모니터링할 수 있는 대시보드를 구성했습니다. 총 문서 수, 검색 가능 청크 수, 벡터 임베딩 상태, OpenAI 연동 상태를 4개 컬럼으로 나누어 직관적으로 표시하며, delta 값을 통해 처리 상태와 성과를 명확히 전달합니다.

In [ ]:
# streamlit_dashboard_final.py
def show_system_monitor(rag_system):
    """시스템 모니터 페이지"""
    st.subheader(" 시스템 상태 모니터링")

    # 실시간 통계 조회
    stats = rag_system.get_system_stats()

    # 메트릭 표시
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.metric(
            label="총 문서 수",
            value=stats['metadata_store']['total_documents'],
            delta="100개 파일 처리 완료"
        )

    with col2:
        st.metric(
            label="검색 가능 청크",
            value=f"{stats['metadata_store']['total_chunks']:,}",
            delta="1,774개 청크 생성"
        )

    with col3:
        st.metric(
            label="벡터 임베딩",
            value=f"{stats['vector_store']['total_chunks']:,}",
            delta="임베딩 완료"
        )

    with col4:
        st.metric(
            label="OpenAI 연동",
            value="정상" if stats['openai_enabled'] else "비활성",
            delta="GPT-4o, text-embedding-3-large"
        )

### 인터랙티브 검색 인터페이스: 다중 검색 방식과 결과 시각화
사용자 친화적인 검색 인터페이스를 위해 텍스트 입력, 선택박스, 스피너 등 Streamlit 위젯을 활용했습니다. 하이브리드, 벡터, 키워드 검색 방식을 선택할 수 있고, 검색 결과는 답변, 신뢰도, 참조 문서로 구조화하여 표시합니다. st.expander()를 통해 참조 문서의 상세 내용을 확인할 수 있어 답변의 근거를 투명하게 제공합니다.

In [ ]:
# streamlit_dashboard_final.py
def show_smart_search(rag_system):
    """스마트 검색 페이지"""
    query = st.text_input(
        " 질문을 입력하세요",
        placeholder="예: 시스템 구축 예산은 얼마인가요?",
        help="자연어로 질문하면 관련 RFP 문서에서 답변을 찾아드립니다."
    )

    # 검색 방식 선택
    search_method = st.selectbox(
        "검색 방식",
        ["hybrid", "vector", "keyword"],
        format_func=lambda x: {
            "hybrid": " 하이브리드 검색 (의미+키워드)",
            "vector": " 의미 검색 (벡터)",
            "keyword": " 키워드 검색 (BM25)"
        }[x]
    )

    if st.button("검색 실행", type="primary"):
        with st.spinner("검색 중..."):
            result = rag_system.search_and_answer(
                query,
                search_method=search_method,
                top_k=5
            )

        # 답변 표시
        st.markdown("###  답변")
        st.markdown(result['answer'])

        # 신뢰도 표시
        confidence_pct = result['confidence'] * 100
        st.markdown(f"**신뢰도**: {confidence_pct:.1f}%")

        # 참조 문서 표시
        if result['sources']:
            st.markdown("###  참조 문서")
            for i, source in enumerate(result['sources']):
                with st.expander(f" {source['metadata']['file_name']}"):
                    st.markdown(source['content_preview'])

In [ ]:
# 향후 구현 예정: 표 데이터 구조화 처리
def extract_and_process_tables(self, document_path: str):
    """표 데이터 추출 및 AI 인식 가능한 형태로 변환"""
    
    # 1. 표 추출 (camelot, tabula 등 활용)
    tables = camelot.read_pdf(document_path, pages='all')
    
    structured_tables = []
    for table in tables:
        # 2. 표를 구조화된 텍스트로 변환
        table_text = self._convert_table_to_structured_text(table.df)
        
        # 3. 표 전용 청크 생성
        table_chunk = DocumentChunk(
            content=table_text,
            metadata={
                'content_type': 'table',
                'table_index': len(structured_tables),
                'columns': list(table.df.columns),
                'row_count': len(table.df)
            },
            chunk_id=str(uuid.uuid4()),
            document_id=self.document_id
        )
        structured_tables.append(table_chunk)
    
    return structured_tables

def _convert_table_to_structured_text(self, df):
    """표를 AI가 이해하기 쉬운 구조화된 텍스트로 변환"""
    structured_text = "표 데이터:\\n"
    structured_text += f"컬럼: {', '.join(df.columns)}\\n"
    
    for idx, row in df.iterrows():
        row_text = " | ".join([f"{col}: {val}" for col, val in row.items()])
        structured_text += f"행 {idx+1}: {row_text}\\n"
    
    return structured_text

---

## 6. 개선방향

최근 RAG 분야의 Survey 논문을 통해 현재 시스템의 개선 방향을 도출했습니다. 현재 구현된 기본 RAG 구조에서 더 나아가, 고도화된 기법들을 단계적으로 적용할 계획입니다.

### 1: 구조화 데이터 처리 강화 - 표 데이터 인식 및 청킹
현재 시스템에서는 텍스트 위주로 처리하지만, RFP 문서에 포함된 표 데이터는 중요한 정보를 담고 있습니다. 앞으로 표 구조를 AI가 인식할 수 있는 형태로 변환하여 별도 청크로 등록하고, 표 내용에 대한 질의응답이 가능하도록 개선할 예정입니다.

### 2: 질의 개선 - LLM 기반 질의 표준화
사용자가 입력하는 질의를 검색에 더 적합하도록 LLM을 활용해 사전 처리합니다. 불명확한 사업명을 정확한 용어로 변경하고, 발주기관명을 표준화된 형태로 수정하며, 기술 용어를 더 구체적으로 명시하여 검색 정확도를 향상시킵니다. 예를 들어 "AI 시스템"을 "인공지능 기반 정보시스템"으로 확장하거나, "공단"을 "국민연금공단" 등 구체적 기관명으로 명시합니다.

### 3: 반복 검색 - 다단계 정보 수집 전략
단일 검색으로 충분한 정보를 얻지 못할 때, 검색 결과를 바탕으로 질의를 개선하여 반복적으로 검색하는 전략입니다. 첫 번째 검색 결과에서 핵심 키워드를 추출하여 두 번째 질의를 생성하고, 이를 통해 더 깊이 있는 정보를 수집합니다. 최대 3회까지 반복하며, 각 단계의 결과를 종합하여 최종 답변의 완성도를 높입니다.

### 4: SQL 기반 스마트 필터링 - 메타데이터 활용 사전 검색 최적화
자연어 질의에서 예산 범위, 마감일, 발주기관 등의 조건을 자동 추출하여 SQL 기반 사전 필터링을 수행합니다. "10억 이상 국민연금공단 사업"과 같은 질의에서 예산 조건과 기관명을 파싱하여 해당 조건을 만족하는 문서만을 대상으로 벡터 검색을 수행함으로써 검색 효율성과 정확도를 크게 향상시킵니다.

### 5: 정밀 출처 표시 - 페이지 번호 기반 학술적 인용
현재 시스템에서는 파일명 수준의 출처 정보만 제공하지만, 앞으로는 PDF 페이지 번호까지 포함한 정밀한 인용 시스템을 구축할 예정입니다. 각 청크에 페이지 위치 정보를 매핑하고, 답변 생성 시 "시스템 구축비는 50억원입니다. [국민연금공단_RFP.pdf, p.15]" 형태로 구체적인 출처를 명시하여 사용자의 검증 편의성을 높입니다.

### 6: 후처리 최적화 - 다층 결과 정제 시스템
검색 결과의 품질을 한층 더 높이기 위해 3단계 후처리 파이프라인을 구축합니다. 

**1) Re-Ranking**: Cross-Encoder 모델을 활용해 질의-문서 쌍의 관련성을 정밀 재평가하여 순위를 재조정합니다.

**2) Redundancy Filtering**: Jaccard 유사도 기반으로 중복성 높은 결과를 제거하여 다양한 관점의 정보를 제공합니다.

**3) Context Enhancement**: 선택된 청크의 전후 맥락을 추가하여 더 완전한 정보를 제공합니다.

이러한 후처리를 통해 최종 사용자에게 제공되는 검색 결과의 관련성, 다양성, 완성도를 모두 향상시킵니다.

### 7: 오픈소스 LLM 대안 모델 분석

**1) EEVE-Korean-10.8B-v1.0**: 한국어 특화 높은 품질 모델로, 모델 크기가 크고 gpu가 24gb 이상 권장됨

**2) Ollama + Llama-3.1-8B**: 설치가 간단하고 cpu 실행 가능하나 성능이 제한적으로 평가됨
